When exploring a large set of documents -- such as Wikipedia, news articles, StackOverflow, etc. -- it can be useful to get a list of related material. To find relevant documents you typically
* Decide on a notion of similarity
* Find the documents that are most similar 

In the assignment you will
* Gain intuition for different notions of similarity and practice finding similar documents. 
* Explore the tradeoffs with representing documents using raw word counts and TF-IDF
* Explore the behavior of different distance metrics by looking at the Wikipedia pages most similar to President Obama’s page.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

## Load Wikipedia dataset

We will be using the dataset of abridged Wikipedia pages. Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase).  

In [ ]:
wiki = pd.read_csv('people_wiki.csv')
wiki.head()

If you want to check whether the text on the webpage agrees with the one here, you can display it with the following code:

In [ ]:
# from IPython.display import HTML
# print(wiki['text'][0])
# HTML(url=wiki['URI'][0])

## Ex. 1: Extract word count vectors

As we have seen in Assignment 4, we can extract word count vectors using `CountVectorizer` function.
- make sure you include words of unit length by using the parameter: `token_pattern=r"(?u)\b\w+\b"`
- do not use any stopwords
- take 10000 most frequent words in the corpus
- explicitly take all the words independent of in how many documents they occur
- obtain the matrix of word counts

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None, max_features=10000)
WCmatrix = vectorizer.fit_transform(wiki.text)


## Ex. 2: Find nearest neighbors

**a)** Start by finding the nearest neighbors of the Barack Obama page using the above word count matrix to represent the articles and **Euclidean** distance to measure distance.
Save the distances in `wiki['BO-eucl']` and look at the top 10 nearest neighbors.

In [ ]:
# One can use the following:
    # from sklearn.neighbors import NearestNeighbors
    # nbrs = NearestNeighbors(n_neighbors=3, algorithm='brute',metric='euclidean').fit(X.toarray())
    # distances, indices = nbrs.kneighbors(X.toarray())
# but here let's use:
from sklearn.metrics import pairwise_distances

obama = WCmatrix[wiki[wiki.name == "Barack Obama"].index]
dist = pairwise_distances(obama, WCmatrix)

wiki["BO-eucl"] = dist.T
wiki.sort_values(by='BO-eucl').head(10)


Showing top ten nearest neighbours to Obamas's page. The pairwise_distances() function computes the distance matrix between two arrays where one is Obama's page and the other is the rest of the pages.

**b)** Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

In [ ]:
obama_bush = WCmatrix[wiki[(wiki.name == "Barack Obama") | (wiki.name == "George W. Bush")].index]
obama_biden = WCmatrix[wiki[(wiki.name == "Barack Obama") | (wiki.name == "Joe Biden")].index]
bush_biden = WCmatrix[wiki[(wiki.name == "George W. Bush") | (wiki.name == "Joe Biden")].index]

distObamaBush = pairwise_distances(obama_bush)
distObamaBiden = pairwise_distances(obama_biden)
distBushBiden = pairwise_distances(bush_biden)

print("Distance Obama - Bush:", distObamaBush[0][1])
print("Distance Obama - Biden:", distObamaBiden[0][1])
print("Distance Bush - Biden:", distBushBiden[0][1])

The smallest distance is between Bush's and Biden's pages.

All of the 10 people from **a)** are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians, e.g.,

* Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
* Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

**c)** Let's find out why Francisco Barrio was considered a close neighbor of Obama.
To do this, look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages.

In [ ]:
def top_words(name):
    df = pd.DataFrame(wiki[wiki.name == name].text.values[0].split(), columns=["word"])
    count = df.word.value_counts()
    df = count.to_frame("count")
    return df.sort_values(by="count", ascending=False)

In [ ]:
obama_words = top_words('Barack Obama')
obama_words

In [ ]:
barrio_words = top_words('Francisco Barrio')
barrio_words

Ten most frequent Obama's and Barrio's words.

**d)** Extract the list of most frequent **common** words that appear in both Obama's and Barrio's documents and display the five words that appear most often in Barrio's article.

Use a dataframe operation known as **join**. The **join** operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the index column of words). See [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) for more details.

In [ ]:
common_words = obama_words.join(barrio_words, lsuffix='_Obama', rsuffix='_Barrio')
common_words.sort_values(by='count_Barrio', ascending=False).head(5)

Collect all words that appear both in Barack Obama and George W. Bush pages.  Out of those words, find the 10 words that show up most often in Obama's page. 

In [ ]:
bush_words = top_words('George W. Bush')
obama_words.join(bush_words, lsuffix='_Obama', rsuffix='_Bush').sort_values(by='count_Obama', ascending=False).dropna().head(10)

**Note.** Even though common words are swamping out important subtle differences, commonalities in rarer political words still matter on the margin. This is why politicians are being listed in the query result instead of musicians, for example. In the next subsection, we will introduce a different metric that will place greater emphasis on those rarer words.

**e)** Among the words that appear in both Barack Obama and Francisco Barrio, take the 15 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 15 words? Which are they?

In [ ]:
common_words = obama_words.join(barrio_words, rsuffix='_Barrio').dropna().sort_values(by='count', ascending=False).head(15)
common_words = set(common_words.index)

articles = []
for enum, text in enumerate(wiki.text):
    if set(text.split()).intersection(common_words) == common_words: 
        articles.append(enum)

print(len(articles))

In [ ]:
wiki.name[articles]

30 articles in the dataset contain all 15 words and those articles are listed above.

## Ex. 3: TF-IDF to the rescue

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So nearest neighbors is recommending plausible results sometimes for the wrong reasons.

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF** (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common.

**a)** Repeat the search for the 10 nearest neighbors of Barack Obama with Euclidean distance of TF-IDF. This time do not limit to only 10000 most frequent words, but take all of them.

In [ ]:
# We could use:
    # from sklearn.feature_extraction.text import TfidfVectorizer
# but since we already know how to compute CountVectorizer, let's use:
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None)
WCmatrix = vectorizer.fit_transform(wiki.text)

tfidf = TfidfTransformer(smooth_idf=False, norm=None)
TFIDFmatrix = tfidf.fit_transform(WCmatrix)

In [ ]:
# now recompute the distances as before but for TF-IDF
dist = pairwise_distances(TFIDFmatrix[wiki[wiki.name == "Barack Obama"].index], TFIDFmatrix)
# add the distances as a column in the wiki dataframe
wiki['BO-eucl-TF-IDF'] = dist.T
wiki[['name', 'BO-eucl-TF-IDF']].sort_values(by='BO-eucl-TF-IDF').head(10)

Top ten nearest neighbours to Obamas's page with Euclidean distance of TF-IDF.

Let's determine whether this list makes sense.
* With a notable exception of Nathan Cullen, the other 8 are all American politicians who are contemporaries of Barack Obama.
* Phil Schiliro, Jesse Lee, Samantha Power, Eric Stern, Eric Holder worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document.

**b)** Sort the words in Obama's article by their TF-IDF weights; do the same for Schiliro's article as well.
Using the **join** operation we learned earlier, compute the common words shared by Obama's and Schiliro's articles.
Sort the common words by their TF-IDF weights in Obama's document.

In [ ]:
def top_words_tf_idf(name):
    """
    Get a table of the largest tf-idf words in the given person's wikipedia page.
    """
    words = wiki[wiki.name == name]['text'].values[0].split()
    ind = {v : i for i, v in enumerate(vectorizer.get_feature_names_out())}
    weigths = [TFIDFmatrix[wiki[wiki.name == name].index][0, ind[word]] for word in words]
    df = pd.Series(weigths, words).to_frame("tf-idf")
    df.drop_duplicates(inplace=True)
    
    return df.sort_values(by="tf-idf", ascending=False)
    

In [ ]:
obama_tf_idf = top_words_tf_idf('Barack Obama')
schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
common_words = obama_tf_idf.join(schiliro_tf_idf, lsuffix='_Obama',rsuffix="_Schiliro")
common_words.dropna().sort_values(by='tf-idf_Obama', ascending=False).head(15)

Top 15 words from Obama's and Schiliro articles sorted by TF-IDF weigth in Obama's document.

**c)** Among the words that appear in both Barack Obama and Phil Schiliro, take the 15 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 15 words? Which are they?

In [ ]:
# It might be helpful to use:
word_to_ind={v: i for i, v in enumerate(vectorizer.get_feature_names())} # a dictionary with words as keys and indices as values

# Your code goes here

common_words = obama_tf_idf.join(schiliro_tf_idf, rsuffix="_Barrio")
obama_common = common_words.dropna().sort_values(by="tf-idf", ascending=False).head(15)
articles = WCmatrix[:, [word_to_ind[word] for word in obama_common.index]]
articles = [i for i, article in enumerate(articles) if article.todense().all()]
print(len(articles))

In [ ]:
wiki.name[articles]

Tree articles in the Wikipedia dataset which contain all of those 15 words with biggest weigths in Obama's document.

Notice the huge difference in this calculation using TF-IDF scores instead  of raw word counts. We've eliminated noise arising from extremely common words.

## Ex. 4: Choosing metrics

**a)** Compute the Euclidean distance between TF-IDF features of Obama and Biden.

In [ ]:
dist = pairwise_distances(TFIDFmatrix[wiki[(wiki.name == "Barack Obama") | (wiki.name == "Joe Biden")].index])
print("TF-IDF Distance between Barack Obama and Joe Biden:", dist[0][1])

The distance is larger than the distances we found for the 10 nearest neighbors, which we repeat here for readability:

In [ ]:
wiki.sort_values(by='BO-eucl-TF-IDF',ascending=True)[['name','BO-eucl-TF-IDF']][0:10]

But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones.

**b)** Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page. To compute text length use the same splitting rules you used in `vectorizer`.

In [ ]:
def compute_length(row):
    return len(row.split())

wiki['length'] = [compute_length(row) for row in wiki.text]

In [ ]:
nearest_neighbors_euclidean = wiki.sort_values(by='BO-eucl-TF-IDF', ascending=True)[['name', 'length', 'BO-eucl-TF-IDF']].head(100)
nearest_neighbors_euclidean

**c)** To see how these document lengths compare to the lengths of other documents in the corpus, make a histogram of the document lengths of Obama's 100 nearest neighbors and compare to a histogram of document lengths for all documents.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.axvline(wiki[wiki.name == "Barack Obama"].length.values[0], label='Length of Barack Obama', color='black', linestyle='--', linewidth=4)
plt.axvline(wiki[wiki.name == "Joe Biden"].length.values[0], label='Length of Joe Biden', color='green', linestyle='--', linewidth=4)
plt.hist(wiki.length, range=[0, 1000], density=True, color='black', label='Entire Wikipedia')
plt.hist(nearest_neighbors_euclidean.length, bins=50, alpha=0.9, density=True, color='red', label='100 NNs of Obama (Euclidean)')

plt.xlabel("# of words")
plt.ylabel("Percentage")
plt.yticks(np.arange(0.00, 0.041, 0.01))
plt.legend()
plt.tight_layout()
plt.show()

Relative to the rest of Wikipedia, nearest neighbors of Obama are overwhemingly short, most of them being shorter than 300 words. The bias towards short articles is not appropriate in this application as there is really no reason to  favor short articles over long articles (they are all Wikipedia articles, after all). Many of the Wikipedia articles are 300 words or more, and both Obama and Biden are over 300 words long.

**Note**: For the interest of computation time, the dataset given here contains _excerpts_ of the articles rather than full text. For instance, the actual Wikipedia article about Obama is around 25000 words. Do not be surprised by the low numbers shown in the histogram.

**Note:** Both word-count features and TF-IDF are proportional to word frequencies. While TF-IDF penalizes very common words, longer articles tend to have longer TF-IDF vectors simply because they have more words in them.

To remove this bias, we turn to **cosine distances**:
$$
d(\mathbf{x},\mathbf{y}) = 1 - \frac{\mathbf{x}^T\mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}
$$
Cosine distances let us compare word distributions of two articles of varying lengths.

**d)** Train a new nearest neighbor model, this time with cosine distances.  Then repeat the search for Obama's 100 nearest neighbors and make a plot to better visualize the effect of having used cosine distance in place of Euclidean on our TF-IDF vectors.

In [ ]:
cosine_dist = pairwise_distances(TFIDFmatrix[wiki[wiki.name == "Barack Obama"].index], TFIDFmatrix, metric="cosine")
wiki["BO-cosine-TF-IDF"] = cosine_dist.T
nearest_neighbors_cosine = wiki.sort_values(by='BO-cosine-TF-IDF', ascending=True)[['name', 'length', 'BO-cosine-TF-IDF']].head(100)
nearest_neighbors_cosine

Obama's 100 nearest neighbors computed with cosine distances.

From a glance at the above table, things look better.  For example, we now see Joe Biden as Barack Obama's nearest neighbor!  We also see Hillary Clinton on the list.  This list looks even more plausible as nearest neighbors of Barack Obama.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.axvline(wiki[wiki.name == "Barack Obama"].length.values[0], label='Barack Obama', color='black', linestyle='--', linewidth=4)
plt.axvline(wiki[wiki.name == "Joe Biden"].length.values[0], label='Joe Biden', color='green', linestyle='--', linewidth=4)
plt.hist(wiki.length, range=[0, 1000], density=True, color='black', label='Entire Wikipedia')
plt.hist(nearest_neighbors_euclidean.length, bins=50, alpha=0.9, density=True, color='red', label='100 NNs of Obama (Euclidean)')
plt.hist(nearest_neighbors_cosine.length, bins=50, alpha=0.8, density=True, color='blue', label='100 NNs of Obama (cosine)')

plt.xlabel("# of words")
plt.ylabel("Percentage")
plt.yticks(np.arange(0.00, 0.041, 0.01))
plt.xlim([0, 1000])
plt.legend()
plt.tight_layout()
plt.show()

Indeed, the 100 nearest neighbors using cosine distance provide a sampling across the range of document lengths, rather than just short articles like Euclidean distance provided.

**Moral of the story**: In deciding the features and distance measures, check if they produce results that make sense for your particular application.

## Ex. 5: Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

```
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+
```

**a)** Transform the tweet into TF-IDF features, using the fit to the Wikipedia dataset. (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.) How similar is this tweet to Barack Obama's Wikipedia article? 

In [ ]:
df = pd.DataFrame({'text': ['democratic governments control law in response to popular act']})

TF_IDF = tfidf.transform(vectorizer.transform(df.text))

def top_words_tf_idf(df):
    words = df.text.values[0].split()
    ind = {v : i for i, v in enumerate(vectorizer.get_feature_names_out())}
    words_weigths = [TF_IDF[0, ind[word]] for word in words]
    words_count = [words.count(word) for word in words]
    data = {"word count" : words_count, "tf_idf" : words_weigths}
    df = pd.DataFrame(data, index=words)
    return df
top_words_tf_idf(df)

Let's compare this tweet's TF-IDF vectors  to Barack Obama's Wikipedia entry.

In [ ]:
obama_tf_idf

**b)** Now, compute the cosine distance between the Barack Obama article and this tweet:

In [ ]:
from sklearn.metrics.pairwise import cosine_distances # for one pair of samples we can just use this function

cosine_distances(TFIDFmatrix[wiki[wiki.name == "Barack Obama"].index], tfidf)

Let's compare this distance to the distance between the Barack Obama article and all of its Wikipedia nearest neighbors:

In [ ]:
nearest_neighbors_cosine[0:23]

With cosine distances, the tweet is "nearer" to Barack Obama than most people! If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet?
In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from _The Atlantic_, you wouldn't recommend him/her a tweet.